Aşağıda **Online Retail** datasetini yükləmək, data understanding, data preparation mərhələlərini keçmək və apriori alqoritmini tətbiq edərək nəticələri göstərmək üçün nümunə kodu təqdim edirəm.

### 1. Datasetin Yüklənməsi

```python
import pandas as pd

# Dataseti yükləmək
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
df = pd.read_excel(url)

# Datasetin ilk 5 sətirini göstərmək
print(df.head())
```

### 2. Data Understanding


**Online Retail Dataset**-in dəyişənlərini aşağıdakı kimi izah edə bilərik:

1. **InvoiceNo**: Alış-verişin unikal identifikatoru. Hər bir satış üçün fərqli bir nömrə.

2. **StockCode**: Məhsulun unikal identifikatoru. Hər bir məhsul üçün fərqli bir kod.

3. **Description**: Məhsulun adı və ya təsviri.

4. **Quantity**: Alınan məhsulun sayı. Negativ dəyərlər, məhsulun qaytarıldığını göstərir.

5. **InvoiceDate**: Satışın tarixi və vaxtı.

6. **UnitPrice**: Məhsulun bir ədədi üçün qiyməti.

7. **CustomerID**: Müştərinin unikal identifikatoru. Müştərinin kim olduğunu müəyyən edir.

8. **Country**: Müştərinin yaşadığı ölkə. Satışların hansı ölkələrdə həyata keçirildiyini göstərir.

Bu dəyişənlər, müştəri davranışlarını, məhsul alqı-satqısını və satışların coğrafi bölgüsünü analiz etmək üçün istifadə edilə bilər. Dataseti daha yaxşı başa düşmək və müxtəlif analitik tapşırıqları yerinə yetirmək üçün bu dəyişənlərdən yararlana bilərsiniz. Hər hansı bir sualınız varsa, kömək etməyə hazıram!

### Data Understanding

Datasetin strukturu və tərkibi haqqında məlumat əldə edirik.

```python
# Datasetin məlumatları haqqında məlumat
print(df.info())

# Unikal müştəri ID-lərini saymaq
print("Unikal müştəri sayı:", df['CustomerID'].nunique())

# Datasetdəki boş dəyərlərin sayı
print(df.isnull().sum())

# Datasetdəki məhsul sayı
print("Məhsul sayı:", df['StockCode'].nunique())

# Müştərilərin alış-veriş tarixlərini yoxlamaq
print(df['InvoiceDate'].describe())
```

### 3. Data Preparation

Boş dəyərləri çıxarırıq və yalnız alış-veriş edən müştəriləri saxlayırıq.

```python
# Boş dəyərləri silmək
df.dropna(subset=['CustomerID'], inplace=True)

# Negativ məhsul miqdarı olan satırları silmək
df = df[df['Quantity'] > 0]

# Tarixi ayırıb gün, ay, il formatında çevirmək
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')

# Məhsul siyahısını yaratmaq
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')

# 1 və 0 formatına çevirmək
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

print(basket.head())
```

### 4. Apriori Modeli Qurmaq

```python
from mlxtend.frequent_patterns import apriori, association_rules

# Apriori alqoritmini tətbiq etmək
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# Qayda çıxarmaq
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Nəticələri göstərmək
print("Tez-tez alınan məhsul birləşmələri:")
print(frequent_itemsets.head())

print("Qayda nəticələri:")
print(rules.head())
```

### 5. Nəticələri Yoxlamaq

Bu mərhələdə apriori nəticələrini yoxlayırıq. Qaydalarda lift, confidence və support dəyərlərinə görə maraqlı olanları seçə bilərsiniz.

```python
# Nəticələri sıralamaq
rules = rules.sort_values(by='lift', ascending=False)

# Yuxarıdan 10 qaydanı göstərmək
print(rules.head(10))
```

### Açıqlama:

- **Data Understanding** mərhələsində datasetin strukturu, boş dəyərləri və müştəri sayı kimi məlumatları öyrənirik.
- **Data Preparation** mərhələsində lazımsız dəyərləri silirik, tarixləri formatlaşdırırıq və alış-veriş səbətlərini hazır vəziyyətə gətiririk.
- **Apriori Alqoritmi** tətbiq edərək müştərilərin birlikdə aldığı məhsul birləşmələrini analiz edirik.

Bu addımları izləyərək datasetdən istifadə edərək apriori alqoritmini tətbiq edə bilərsiniz. Hər hansı bir sualınız varsa, kömək etməkdən məmnunam!

## 1. Online Retail  Datasetin Yüklənməsi

In [1]:
import pandas as pd

# Datasetin yüklənməsi (Excel formatında)
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
df = pd.read_excel(url)

In [2]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


## 2. Data Understanding

## Datasetin məlumatları haqqında məlumat

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


## Unikal müştəri ID-lərini saymaq

In [4]:
print("Unikal müştəri sayı:", df['CustomerID'].nunique())

Unikal müştəri sayı: 4372


## Datasetdəki boş dəyərlərin sayı

In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

## Datasetdəki məhsul sayı

In [6]:
print("Məhsul sayı:", df['StockCode'].nunique())

Məhsul sayı: 4070


## Müştərilərin alış-veriş tarixlərini yoxlamaq

In [7]:
df['InvoiceDate'].describe()

count                           541909
mean     2011-07-04 13:34:57.156386048
min                2010-12-01 08:26:00
25%                2011-03-28 11:34:00
50%                2011-07-19 17:17:00
75%                2011-10-19 11:27:00
max                2011-12-09 12:50:00
Name: InvoiceDate, dtype: object

## 3. Data Preparation

## Boş dəyərləri silmək

In [8]:
df.dropna(subset=['CustomerID'], inplace=True)

## Negativ məhsul miqdarı olan satırları silmək

In [9]:
df = df[df['Quantity'] > 0]

## Tarixi ayırıb gün, ay, il formatında çevirmək

In [10]:
df.loc[:, 'InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.loc[:, 'YearMonth'] = df['InvoiceDate'].dt.to_period('M')

C:\Users\dell\AppData\Local\Temp\ipykernel_8928\2691146516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'YearMonth'] = df['InvoiceDate'].dt.to_period('M')


In [11]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,YearMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011-12
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12


## Məhsul siyahısını yaratmaq

In [12]:

basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().\
                                            unstack().reset_index().fillna(0).set_index('InvoiceNo')

In [13]:
basket

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,24.0,0.0,0.0


## 1 və 0 formatına çevirmək

In [14]:
basket = basket.apply(lambda x: (x > 0).astype(int), axis=1)

In [15]:
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

basket

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PC CUTLERY SET PANTRY DESIGN,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,200 RED + WHITE BENDY STRAWS,3 BIRDS CANVAS SCREEN,3 BLACK CATS W HEARTS BLANK CARD,3 DRAWER ANTIQUE WHITE WOOD CABINET,3 GARDENIA MORRIS BOXED CANDLES,3 HEARTS HANGING DECORATION RUSTIC,3 HOOK HANGER MAGIC GARDEN,3 HOOK PHOTO SHELF ANTIQUE WHITE,3 PIECE SPACEBOY COOKIE CUTTER SET,3 PINK HEN+CHICKS IN BASKET,...,WRAP RED VINTAGE DOILY,WRAP SUKI AND FRIENDS,WRAP SUMMER ROSE DESIGN,WRAP VINTAGE LEAF DESIGN,WRAP VINTAGE PETALS DESIGN,WRAP WEDDING DAY,"WRAP, BILLBOARD FONTS DESIGN","WRAP, CAROUSEL",YELLOW BREAKFAST CUP AND SAUCER,YELLOW COAT RACK PARIS FASHION,YELLOW DRAGONFLY HELICOPTER,YELLOW EASTER EGG HUNT START POST,YELLOW FELT HANGING HEART W FLOWER,YELLOW FLOWERS FELT HANDBAG KIT,YELLOW GIANT GARDEN THERMOMETER,YELLOW METAL CHICKEN HEART,YELLOW PINK FLOWER DESIGN BIG BOWL,YELLOW POT PLANT CANDLE,YELLOW SHARK HELICOPTER,YELLOW/BLUE RETRO RADIO,YELLOW/ORANGE FLOWER DESIGN PLATE,YELLOW/PINK FLOWER DESIGN BIG MUG,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,YULETIDE IMAGES S/6 PAPER BOXES,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FINISH 15CM PLANTER POTS,ZINC FOLKART SLEIGH BELLS,ZINC HEART FLOWER T-LIGHT HOLDER,ZINC HEART LATTICE 2 WALL PLANTER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC PLANT POT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## 4. Apriori Modeli Qurmaq

## Apriori alqoritmini tətbiq etmək

In [16]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

C:\Users\dell\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


MemoryError: Unable to allocate 26.5 GiB for an array with shape (191890, 2, 18536) and data type int32

# Apriori modelini daha az mallar ilə icra etmək
# Məsələn, ilk 50 məhsul seçin

In [31]:
top_items = basket.sum().nlargest(50).index
basket = basket[top_items]

In [32]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

C:\Users\dell\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


## Qayda çıxarmaq

In [33]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

##  Nəticələri göstərmək

In [34]:
print("Tez-tez alınan məhsul birləşmələri:")
frequent_itemsets.head()

Tez-tez alınan məhsul birləşmələri:


,support,itemsets
0,0.106334,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.091929,(REGENCY CAKESTAND 3 TIER)
2,0.086319,(JUMBO BAG RED RETROSPOT)
3,0.074450,(PARTY BUNTING)
4,0.074180,(ASSORTED COLOUR BIRD ORNAMENT)


In [35]:
print("Qayda nəticələri:")
rules.head()

Qayda nəticələri:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(REGENCY CAKESTAND 3 TIER),(WHITE HANGING HEART T-LIGHT HOLDER),0.091929,0.106334,0.012678,0.137911,1.296963,0.002903,1.036629,0.252148
1,(WHITE HANGING HEART T-LIGHT HOLDER),(REGENCY CAKESTAND 3 TIER),0.106334,0.091929,0.012678,0.119229,1.296963,0.002903,1.030995,0.256212
2,(JUMBO BAG RED RETROSPOT),(WHITE HANGING HEART T-LIGHT HOLDER),0.086319,0.106334,0.013541,0.156875,1.475309,0.004363,1.059945,0.352613
3,(WHITE HANGING HEART T-LIGHT HOLDER),(JUMBO BAG RED RETROSPOT),0.106334,0.086319,0.013541,0.127347,1.475309,0.004363,1.047015,0.360511
4,(PARTY BUNTING),(WHITE HANGING HEART T-LIGHT HOLDER),0.074450,0.106334,0.015052,0.202174,1.901317,0.007135,1.120127,0.512180


## 5. Nəticələri Yoxlamaq
## Bu mərhələdə apriori nəticələrini yoxlayırıq. Qaydalarda lift, confidence və support dəyərlərinə görə maraqlı olanları seçə bilərsiniz.

##  Nəticələri sıralamaq

In [36]:
rules = rules.sort_values(by='lift', ascending=False)

In [37]:
# Yuxarıdan 10 qaydanı göstərmək

In [38]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
206,"(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT)","(LUNCH BAG PINK POLKADOT, LUNCH BAG BLACK SKU...",0.024709,0.025032,0.010412,0.421397,16.834099,0.009794,1.685038,0.964426
203,"(LUNCH BAG PINK POLKADOT, LUNCH BAG BLACK SKU...","(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT)",0.025032,0.024709,0.010412,0.415948,16.834099,0.009794,1.669871,0.964747
205,"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)","(LUNCH BAG PINK POLKADOT, LUNCH BAG RED RETROS...",0.022713,0.028215,0.010412,0.458432,16.247612,0.009771,1.794392,0.960263
204,"(LUNCH BAG PINK POLKADOT, LUNCH BAG RED RETROS...","(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)",0.028215,0.022713,0.010412,0.369025,16.247612,0.009771,1.548853,0.965700
202,"(LUNCH BAG PINK POLKADOT, LUNCH BAG CARS BLUE)","(LUNCH BAG BLACK SKULL., LUNCH BAG RED RETROS...",0.023036,0.027892,0.010412,0.451991,16.205219,0.009770,1.773890,0.960416
207,"(LUNCH BAG BLACK SKULL., LUNCH BAG RED RETROS...","(LUNCH BAG PINK POLKADOT, LUNCH BAG CARS BLUE)",0.027892,0.023036,0.010412,0.373308,16.205219,0.009770,1.558921,0.965213
208,(LUNCH BAG PINK POLKADOT),"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.,...",0.050227,0.014135,0.010412,0.207304,14.666361,0.009702,1.243686,0.981094
201,"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.,...",(LUNCH BAG PINK POLKADOT),0.014135,0.050227,0.010412,0.736641,14.666361,0.009702,3.606386,0.945177
164,"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)",(LUNCH BAG PINK POLKADOT),0.022713,0.050227,0.014027,0.617577,12.295823,0.012886,2.483569,0.940022
165,(LUNCH BAG PINK POLKADOT),"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)",0.050227,0.022713,0.014027,0.279270,12.295823,0.012886,1.355968,0.967253


# Predict

Apriori alqoritmini tətbiq etdikdən sonra, hansı məhsulların bir yerdə satıldığını görmək üçün `rules` DataFrame-indən istifadə edə bilərsiniz. Bu DataFrame, məhsul birləşmələrinin əlaqələrini (qaydalarını) ehtiva edir. Aşağıdakı kodu istifadə edərək, müəyyən bir məhsulun yanında hansı məhsulların daha çox satıldığını görə bilərsiniz:

1. **Məhsul adını müəyyən edin**: Məsələn, "Product A" məhsulunun yaninda hansı məhsulların satıldığını öyrənmək istədiyinizi düşünək.

2. **Qaydalarda məhsulu axtarın**: Aşağıdakı kodu istifadə edin:

```python
# Məhsul adı
product_name = 'Product A'

# Məhsulun yaninda satılan digər məhsulları tapmaq
product_rules = rules[(rules['antecedents'].apply(lambda x: product_name in x)) | 
                      (rules['consequents'].apply(lambda x: product_name in x))]

print("Məhsul yaninda satılan məhsullar:")
print(product_rules)
```

Bu kod, "Product A" məhsulunun həm "antecedents" (yəni, "Product A" satışını təsdiq edən məhsullar), həm də "consequents" (yəni, "Product A" satıldıqda satılan məhsullar) daxil olduğu bütün qaydaları göstərəcək. 

Bu şəkildə, konkret bir məhsulun yaninda hansı məhsulların daha çox alındığını asanlıqla görə bilərsiniz. İstədiyiniz başqa bir məhsul varsa, sadəcə `product_name` dəyərini dəyişdirməyiniz kifayətdir. Hər hansı bir sualınız varsa, kömək etməkdən məmnunam!

In [39]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,YearMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011-12
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12


In [40]:
# Məhsul adı
product_name = 'WHITE HANGING HEART T-LIGHT HOLDER'

# Məhsulun yaninda satılan digər məhsulları tapmaq
product_rules = rules[(rules['antecedents'].apply(lambda x: product_name in x)) | 
                      (rules['consequents'].apply(lambda x: product_name in x))]

print("Məhsul yaninda satılan məhsullar:")
product_rules

Məhsul yaninda satılan məhsullar:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
19,(WHITE HANGING HEART T-LIGHT HOLDER),(HEART OF WICKER SMALL),0.106334,0.051845,0.014458,0.135972,2.622653,0.008945,1.097365,0.692324
18,(HEART OF WICKER SMALL),(WHITE HANGING HEART T-LIGHT HOLDER),0.051845,0.106334,0.014458,0.278876,2.622653,0.008945,1.239269,0.652538
15,(WHITE HANGING HEART T-LIGHT HOLDER),(NATURAL SLATE HEART CHALKBOARD ),0.106334,0.052169,0.014297,0.134450,2.577204,0.008749,1.095062,0.684800
14,(NATURAL SLATE HEART CHALKBOARD ),(WHITE HANGING HEART T-LIGHT HOLDER),0.052169,0.106334,0.014297,0.274043,2.577204,0.008749,1.231019,0.645666
21,(WHITE HANGING HEART T-LIGHT HOLDER),(LUNCH BAG PINK POLKADOT),0.106334,0.050227,0.010682,0.100457,2.000069,0.005341,1.055839,0.559512
20,(LUNCH BAG PINK POLKADOT),(WHITE HANGING HEART T-LIGHT HOLDER),0.050227,0.106334,0.010682,0.212675,2.000069,0.005341,1.135066,0.526459
5,(WHITE HANGING HEART T-LIGHT HOLDER),(PARTY BUNTING),0.106334,0.074450,0.015052,0.141553,1.901317,0.007135,1.078168,0.530454
4,(PARTY BUNTING),(WHITE HANGING HEART T-LIGHT HOLDER),0.074450,0.106334,0.015052,0.202174,1.901317,0.007135,1.120127,0.512180
7,(WHITE HANGING HEART T-LIGHT HOLDER),(ASSORTED COLOUR BIRD ORNAMENT),0.106334,0.074180,0.014620,0.137494,1.853515,0.006732,1.073407,0.515276
6,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.074180,0.106334,0.014620,0.197091,1.853515,0.006732,1.113036,0.497380
